## 0. Libarary 불러오기 및 경로설정

In [2]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from glob import glob
import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from torch.utils.tensorboard import SummaryWriter

In [15]:
# A = torch.tensor([[100,1000,1],[1,1,10]])
# b = torch.argmax(A, dim=1) 
# c = torch.tensor([0,2])
# torch.tensor(b==c)


/tmp/ipykernel_40057/2434658839.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(b==c)


tensor([False,  True])

## test관련 세팅


In [18]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = os.path.join(os.getcwd() , 'datasets/eval/')
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID] #imgDirList
# 이미지 주소 리스트를 반환해줌

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(), #Data Augmentation 데이터 증진?!
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225],
    ),
])




In [3]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), #Data Augmentation 데이터 증진?!
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225],
    ),
])


## 1. Model 정의

In [12]:


gender_classifier = torchvision.models.resnet34(pretrained=True)
gender_classifier.fc = torch.nn.Sequential(
    nn.Linear(in_features=512, out_features=2, bias=True),
    nn.Sigmoid(),
)#age는 선형회귀로 만들어보기



<h1 style="color='blue'">Train Dataset 정의</h1>

In [4]:
class Train_Datasets (torch.utils.data.Dataset):
    _train_csv = pd.read_csv('./datasets/train/train.csv')
    dir_list=  glob('./datasets/train/images/*/*')
    

    def __len__ (self):
        return self.dir_list.__len__()
    
    def __getitem__ (self , idx):
        dir = self.dir_list[idx],
        _wear = ''
        if self.dir_list[idx].split('/')[-1].startswith('mask') :
            _wear = 2
        elif self.dir_list[idx].split('/')[-1].startswith('inc'):
            _wear = 1
        else : 
            _wear = 0
        id, gender, _ , age = self.dir_list[idx].split('/')[-2].split('_')
        gender = 0 if gender == 'female' else 1
        img = Image.open(self.dir_list[idx]) #해당 주소로 이미지 열고
        X = transforms_train(img)
        age = int(age)
        if age <30 :
            age = 0 
        elif age >= 30 and age <60 :
            age =1
        else:
            age=2

        #.unsqueeze(0)  if transforms_train(img).shape.__len__() == 3 else  transforms_train(img)# 트랜스폼에 전달해서 텐서화
        y = (
            torch.tensor(gender), 
            torch.tensor(_wear), 
            torch.tensor(age)
        ) #y는 우리가 모델로 판별하고자 하는 요소들, 젠더, 마스크착용형태, age
        return X, y

# y는 gender , 마스크 착용여부, 나이 
# gender는 여자가 0, 남자가 1
# 마스크는 제대로 쓰면 2, 이상하게 쓰면 1, 안쓰면 ,0
        

In [5]:
train_datasets = Train_Datasets()
train_databatch = torch.utils.data.DataLoader(
    train_datasets,
    batch_size = 100,
    shuffle = True,
    num_workers = 4,
)

# 1. 마스크 분류모델 모델
### 라벨링
- <mark>[미착용, 오착용, 정상착용]</mark> 순서로 라벨링 되어 있음.

In [6]:
# 1. 모델의 정의
# 전이학습 모델로 만듦
mask_classifier = torchvision.models.resnet34(pretrained=True)
mask_classifier.fc = nn.Sequential(
    nn.Linear(in_features = 512, out_features = 3, bias=True),
    nn.Softmax(),
)

In [7]:
# 🌊텐서보드 관련
writer = SummaryWriter()

In [ ]:
# 2. 학습모드
epochs = 1
def mask_criterion (pred, y):
    #마스크는 3가지 클래스로 분류.
    #즉 실제 정답 y는 0,1,2 셋 중 하나를 값으로 가짐.
    pred = -1 * torch.log2(pred) # 모델이 예측한 클래스별 확률(=pred) 전체에 로그를 씌움.... 
    pred = 0.025 * pred # 0.025를 일단 곱해주자. 왜냐하면 일종의 라벨 스무딩임...
    # 라벨에 해당되는 원소는 0.95/0.025=38를 곱해주자.
    for idx, label in enumerate(y): #y로 반복문(for)을 돌리자.
        pred[idx,label ] = 38 * pred[idx,label ]
    
    return torch.sum(pred)
    
mask_opti =torch.optim.Adam(mask_classifier.parameters(), lr=0.001)
device = torch.device('cuda')
mask_classifier.to(device).train()

for _i_d_x__ in range(epochs):
    for i, ( X, y ) in enumerate(train_databatch):
        X = X.to(device)
        _ ,mask_real , __ = y
        mask_real = mask_real.to(device)
        print(mask_real)
        mask_opti.zero_grad()
        mask_pred = mask_classifier(X)     
        print('모델 예측 값 ', mask_pred)
        print('-'*50)
        loss_mask = mask_criterion(mask_pred, mask_real)
        print( i, 'mask _loss : ', loss_mask)
        writer.add_scalar(
            "Loss/train", 
            loss_mask, 
            i + train_datasets.__len__()*_i_d_x__
        )
        loss_mask.backward()
        mask_opti.step()
writer.flush()

In [ ]:
!tensorboard --logdir=runs
#writer.close()

In [ ]:
# Test 모드
test_model = mask_classifier 
imglist = glob('./datasets/eval/images/*')
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225],
    ),
])
test_model = test_model.to(device)
test_model.eval()
with torch.no_grad() :
    for idx, dir in enumerate(imglist):
        print(dir)
        if idx < 10 : 
            continue
        if idx == 20 :
            break
        img = Image.open(dir)
        plt.imshow(img)
        img = transforms_test(img).to(torch.device('cuda'))
        age_predict = test_model(img.unsqueeze(0))
        plt.title(age_predict)
        plt.show()

# 2. 연령 분류 모델

In [42]:
age_classifier = torchvision.models.resnet34(pretrained=True)

age_classifier.fc = nn.Sequential(
    nn.Linear(in_features = 512, out_features = 3, bias=True),
    nn.Softmax(),
)

In [ ]:
# 2. 학습모드
epochs = 1
def age_criterion (pred, y):
    #에이지 역시 3가지 클래스로 분류.
    #즉 실제 정답 y는 0,1,2 셋 중 하나를 값으로 가짐.
    pred = -1 * torch.log2(pred) # 모델이 예측한 클래스별 확률(=pred) 전체에 로그를 씌움.... 
    pred = 0.05 * pred # 0.05를 일단 곱해주자. 왜냐하면 일종의 라벨 스무딩임...
    # 라벨에 해당되는 원소는 0.95/0.05=19를 곱해주자.
    for idx, label in enumerate(y): #y로 반복문(for)을 돌리자.
        pred[idx,label ] = 19 * pred[idx,label ]
    
    return torch.sum(pred)
    
age_opti =torch.optim.Adam(age_classifier.parameters(), lr=0.001)
device = torch.device('cuda')
age_classifier.to(device).train()

correct_count = 0
for _ in range(epochs):
    
    for i, (X, y) in enumerate(train_databatch):
        X = X.to(device)
        _ ,__ , age_real = y
        age_real = age_real.to(device)
        
        # print(age_real)
        
        age_opti.zero_grad()
        age_pred = age_classifier(X)   

        how_much = torch.sum(torch.argmax(age_pred, dim=1) == age_real).item()
        print(
            i, 
            "번째 배치에서의 정답률",
            how_much,
            "%",
        )
        correct_count += how_much

        # print('모델 예측 값 ', age_pred)
        # print('-'*50)
        loss_age = age_criterion(age_pred, age_real)
        # print( i, 'mask _loss : ', loss_age)
        loss_age.backward()
        age_opti.step()
print('모델 정확도 : ', correct_count/train_datasets.__len__())

In [ ]:
# Test 모드
test_model = age_classifier 
imglist = glob('./datasets/eval/images/*')
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225],
    ),
])
test_model = test_model.to(device)
test_model.eval()
with torch.no_grad() :
    for idx, dir in enumerate(imglist):
        print(dir)
        if idx < 10 : 
            continue
        if idx == 20 :
            break
        img = Image.open(dir)
        plt.imshow(img)
        img = transforms_test(img).to(torch.device('cuda'))
        age_predict = test_model(img.unsqueeze(0))
        plt.title(age_predict)
        plt.show()

# 3. 젠더 분류 모델

In [28]:
gender_classifier = torchvision.models.resnet34(pretrained=True)

gender_classifier.fc = nn.Sequential(
    nn.Linear(in_features = 512, out_features = 2, bias=True),
    nn.Softmax(),
)

In [ ]:
# 2. 학습모드
epochs = 1
def gender_criterion (pred, y):
    #에이지 역시 3가지 클래스로 분류.
    #즉 실제 정답 y는 0,1,2 셋 중 하나를 값으로 가짐.
    pred = -1 * torch.log2(pred) # 모델이 예측한 클래스별 확률(=pred) 전체에 로그를 씌움.... 
    pred = 0.025 * pred # 0.025를 일단 곱해주자. 왜냐하면 일종의 라벨 스무딩임...
    # 라벨에 해당되는 원소는 0.975/0.025=39를 곱해주자.
    for idx, label in enumerate(y): #y로 반복문(for)을 돌리자.
        pred[idx,label ] = 39 * pred[idx,label ]
    
    return torch.sum(pred)
    
gender_opti =torch.optim.Adam(gender_classifier.parameters(), lr=0.001)
device = torch.device('cuda')
gender_classifier.to(device).train()


correct_count = 0

for _ in range(epochs):
    for i,( X, y ) in enumerate( train_databatch):
        X = X.to(device)
        gender_real ,__ , _ = y
        
        gender_real = gender_real.to(device)
        # print("실제 성별 : " ,gender_real)
        gender_opti.zero_grad()
        gender_pred = gender_classifier(X)    
        

        how_much = torch.sum( torch.argmax(gender_pred, dim=1) == gender_real).item()
        print(
            i, 
            "번째 배치에서의 정답률", 
            how_much,
            '%',
        )

        correct_count += how_much

        # print('모델 예측 값 ', gender_pred)
        # print('-'*50)
        loss_gender = gender_criterion(gender_pred, gender_real)
        print('배치: ', i, '// gender _loss : ', loss_gender.item())
        loss_gender.backward()
        gender_opti.step()

print(
    "모델 정확도 :", 
    correct_count/train_datasets.__len__())

In [ ]:
# Test 모드
test_model = gender_classifier 
imglist = glob('./datasets/eval/images/*')
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225],
    ),
])
test_model = test_model.to(device)
test_model.eval()
with torch.no_grad() :
    for idx, dir in enumerate(imglist):
        print(dir)
        if idx < 10 : 
            continue
        if idx == 20 :
            break
        img = Image.open(dir)
        plt.imshow(img)
        img = transforms_test(img).to(torch.device('cuda'))
        age_predict = test_model(img.unsqueeze(0))
        plt.title(age_predict)
        plt.show()

## 3. Inference

In [29]:
device = torch.device('cuda')

mask_classifier.eval()
gender_classifier.eval()
age_classifier.eval()

all_predictions =[]
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
for img_dir in image_paths:
    img_obj = Image.open(img_dir)
    img_obj = transforms_test(img_obj).unsqueeze(0).to(device)
    with torch.no_grad():
        img_obj = img_obj.to(device)
        pred = model(img_obj)
        pred = pred.argmax(dim=-1)
        pred = pred.cpu().numpy()
        idx_list = [0,1,10,11,12,13,14,15,16,17,2,3,4,5,6,7,8,9]
        pred = idx_list[int(pred)]
        all_predictions.append(pred)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
